In [4]:
import kagglehub
import numpy as np
import pandas as pd

from tabulate import tabulate
import matplotlib.pyplot as plt

from wordcloud import WordCloud
import seaborn as sns
from textblob import TextBlob

from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

import string
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Ensure NLTK resources are downloaded
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...


True

In [5]:
# Download latest version
path = kagglehub.dataset_download("saurabhshahane/fake-news-classification")

print("Path to dataset files:", path)

# load datasets
df=pd.read_csv(path+'/WELFake_Dataset.csv')
df.head()

100%|██████████| 92.1M/92.1M [00:08<00:00, 10.8MB/s]

Extracting files...


Path to dataset files: C:\Users\nikhi\.cache\kagglehub\datasets\saurabhshahane\fake-news-classification\versions\77


,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [6]:
df.shape

(72134, 4)

In [7]:
df['article'] = df['title'] + df['text']
df = df[['article', 'label']]
df.head()

,article,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1
1,NaN,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",0
4,SATAN 2: Russia unvelis an image of its terrif...,1


In [8]:
df.dropna(inplace=True)
df.drop_duplicates(subset='article', inplace=True)
df.shape

(63121, 2)

In [9]:
df['sentiment_polarity'] = df['article'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
df['sentiment_subjectivity'] = df['article'].apply(lambda x: TextBlob(str(x)).sentiment.subjectivity)

In [10]:
df.head()

,article,label,sentiment_polarity,sentiment_subjectivity
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1,0.034641,0.296824
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1,0.190402,0.512798
3,"Bobby Jindal, raised Hindu, uses story of Chri...",0,0.111493,0.387106
4,SATAN 2: Russia unvelis an image of its terrif...,1,0.013342,0.403577
5,About Time! Christian Group Sues Amazon and SP...,1,-0.100461,0.428571


# preprocessing

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df['article'], df['label'], test_size=0.2, random_state=42, stratify=df['label'])
overlap = set(X_train).intersection(set(X_test))
print(f"Number of overlapping samples between train and test sets: {len(overlap)}")

Number of overlapping samples between train and test sets: 0


In [12]:
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    text = re.sub(r'\b([a-zA-Z])\.(?=[a-zA-Z])', r'\1specialdot', text)  # Preserve acronyms
    text = re.sub(r'\b([a-zA-Z])\.(?=\s|$)', r'\1specialdot', text)
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = text.replace("specialdot", ".")
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize, remove stopwords, and lemmatize
    tokens = text.split()
    stop_words = set(stopwords.words('english'))
    stop_words.discard("not")
    stop_words.discard("no")
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Vectorize text
def vectorize(text, maxFeats, ngram):
    vectorizer = TfidfVectorizer(max_features=maxFeats, ngram_range=ngram, stop_words='english')
    X_tfidf = vectorizer.fit_transform(text)
    return X_tfidf, vectorizer

# Load data
X_train, X_test, y_train, y_test = train_test_split(
    df['article'], df['label'], 
    test_size=0.2, random_state=42, stratify=df['label']
)

# Preprocess data
X_train_processed = X_train.apply(preprocess_text)
X_test_processed = X_test.apply(preprocess_text)

# Vectorize data
X_train_vectorized, vectorizer = vectorize(X_train_processed, 2000, (1,1))
X_test_vectorized = vectorizer.transform(X_test_processed)

# Train model
#model = RandomForestClassifier()
#model.fit(X_train_vectorized, y_train)

# Predict and evaluate on test set
#y_pred = model.predict(X_test_vectorized)
#print(classification_report(y_test, y_pred))
#print(f"Test Set Accuracy: {accuracy_score(y_test, y_pred)}")

# # Cross-validation
# scores = cross_val_score(model, X_train_vectorized, y_train, cv=5, scoring='accuracy')
# print("Cross-validation Accuracies:", scores)
# print("Average Accuracy:", scores.mean())


In [13]:
X_train_vectorized

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 5681473 stored elements and shape (50496, 2000)>

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

def run_svm_pipeline(X_train, X_test, y_train, y_test):
    """
    Implements an SVM classifier for fake news detection
    """
    # Initialize SVM with a linear kernel
    svm_model = SVC(kernel='linear', probability=True, random_state=42)
    
    # Train the model on the training data
    svm_model.fit(X_train, y_train)
    
    # Predict the test set
    y_pred = svm_model.predict(X_test)
    
    # Calculate accuracy and detailed classification report
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    
    return svm_model, accuracy, report

# Run the SVM pipeline
model, accuracy, report = run_svm_pipeline(X_train_vectorized, X_test_vectorized, y_train, y_test)

# Display accuracy and report:
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", report)


Accuracy: 93.67%
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.94      0.94      6959
           1       0.93      0.93      0.93      5666

    accuracy                           0.94     12625
   macro avg       0.94      0.94      0.94     12625
weighted avg       0.94      0.94      0.94     12625



In [ ]:
# Cross-validation
scores = cross_val_score(model, X_train_vectorized, y_train, cv=5, scoring='accuracy')
print("Cross-validation Accuracies:", scores)
print("Average Accuracy:", scores.mean())

# testing

In [ ]:
# Evaluate on new test data
test_data = [
    {"article": "Breaking news: The President announces a major economic reform to boost GDP growth.", "label": 1},
    {"article": "Scientists claim that the Earth is flat and provide evidence at a recent conference.", "label": 0},
    {"article": "Recent studies confirm the safety and effectiveness of COVID-19 vaccines worldwide.", "label": 1},
    {"article": "A man in Ohio claims he saw an alien spaceship while walking his dog late at night.", "label": 0},
    {"article": "The government has approved new environmental regulations to tackle climate change.", "label": 1},
    {"article": "Fake story circulates on social media claiming that sugar cures all diseases.", "label": 0},
    {"article": "The international summit on renewable energy has successfully concluded in Paris.", "label": 1},
    {"article": "A viral post claims that drinking only water for a month can cure cancer.", "label": 0},
    {"article": "Breaking: NASA confirms the discovery of a new habitable planet in a nearby star system.", "label": 1},
    {"article": "Authorities warn about a rise in fake news websites spreading misinformation online.", "label": 1}
]

# Convert to DataFrame
test_data_df = pd.DataFrame(test_data)

# Preprocess and vectorize test data
test_data_processed = test_data_df['article'].apply(preprocess_text)
test_data_vectorized = vectorizer.transform(test_data_processed)

# Predict labels for test data
y_pred_test = model.predict(test_data_vectorized)

# Evaluate model predictions
print("Predictions vs Ground Truth:")
for text, pred, true in zip(test_data_df['article'], y_pred_test, test_data_df['label']):
    print(f"News: {text}\nPrediction: {'True' if pred == 1 else 'Fake'} | Ground Truth: {'True' if true == 1 else 'Fake'}\n")

# Classification report for test data
print(classification_report(test_data_df['label'], y_pred_test, target_names=["Fake", "True"]))

In [12]:
# Download latest version
path = kagglehub.dataset_download("bhavikjikadara/fake-news-detection")

print("Path to dataset files:", path)

# load datasets
fake=pd.read_csv(path+'/fake.csv')
fake['label'] = 0
true=pd.read_csv(path+'/true.csv')
true['label'] = 1
test = pd.concat([fake, true])
test.head()

100%|██████████| 41.0M/41.0M [00:01<00:00, 23.4MB/s]

Extracting files...


Path to dataset files: C:\Users\ronak\.cache\kagglehub\datasets\bhavikjikadara\fake-news-detection\versions\1


,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [13]:
test['article'] = test['title'] + test['text']
test = test[['article', 'label']]

In [14]:
test.dropna(inplace=True)
test.drop_duplicates(subset='article', inplace=True)
test.shape

(39105, 2)

In [15]:
test['sentiment_polarity'] = test['article'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
test['sentiment_subjectivity'] = test['article'].apply(lambda x: TextBlob(str(x)).sentiment.subjectivity)
test.head()

,article,label,sentiment_polarity,sentiment_subjectivity
0,Donald Trump Sends Out Embarrassing New Year’...,0,0.083136,0.597204
1,Drunk Bragging Trump Staffer Started Russian ...,0,-0.020811,0.344802
2,Sheriff David Clarke Becomes An Internet Joke...,0,-0.012345,0.541969
3,Trump Is So Obsessed He Even Has Obama’s Name...,0,-0.038021,0.413021
4,Pope Francis Just Called Out Donald Trump Dur...,0,-0.011722,0.495222


In [27]:
test['label'] = 1 - test['label']

In [28]:
test.head()

,article,label,sentiment_polarity,sentiment_subjectivity
0,Donald Trump Sends Out Embarrassing New Year’...,1,0.083136,0.597204
1,Drunk Bragging Trump Staffer Started Russian ...,1,-0.020811,0.344802
2,Sheriff David Clarke Becomes An Internet Joke...,1,-0.012345,0.541969
3,Trump Is So Obsessed He Even Has Obama’s Name...,1,-0.038021,0.413021
4,Pope Francis Just Called Out Donald Trump Dur...,1,-0.011722,0.495222


In [29]:
X, y = test['article'], test['label']
X = X.apply(preprocess_text)
X = vectorizer.transform(X)
# Predict and evaluate on test set
# print(classification_report(y, y_pred))
# print(f"Test Set Accuracy: {accuracy_score(y, y_pred)}")

In [31]:

X_test_tensor1 = torch.tensor(X.toarray(),dtype=torch.float32)
y_test_tensor1 = torch.tensor(y.values,dtype=torch.float32)

test_dataset1 = TensorDataset(X_test_tensor1,y_test_tensor1)
test_loader1 = DataLoader(test_dataset1,batch_size=32)


# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader1:
        outputs = model(X_batch).squeeze()
        predictions = (outputs >= 0.5).float()
        total += y_batch.size(0)
        correct += (predictions == y_batch).sum().item()

print(f"Test Accuracy: {correct / total:.4f}")

Test Accuracy: 0.9947
